In [1]:
import transformers
from transformers import pipeline

pipe = pipeline(task='text-classification', model='ProsusAI/finbert')

C:\Users\ivane\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [2]:
total_score = 0
num_articles = 0

In [5]:
import pandas as pd

file_path = fr'C:\Users\ivane\Desktop\Dissertation\data\maltabusinessweeklyArticles.csv'
df = pd.read_csv(fr'{file_path}')

df.head()

,headline,url,description,date
0,"Inflation, environment main concerns for Malte...",https://maltabusinessweekly.com/inflation-envi...,Inflation and the environment are the major co...,2024-12-19
1,16 local councils in financial trouble as they...,https://maltabusinessweekly.com/16-local-counc...,16 local councils are in financial trouble as ...,2024-12-19
2,Parliament approves 2025 Budget estimates for ...,https://maltabusinessweekly.com/parliament-app...,Parliament has officially approved the 2025 Bu...,2024-12-16
3,PwC Malta’s CEO Confidence Tracker reveals gen...,https://maltabusinessweekly.com/pwc-maltas-ceo...,The latest PwC CEO Confidence Tracker which wa...,2024-12-16
4,Court revokes permit to add six floors on Rame...,https://maltabusinessweekly.com/court-revokes-...,Flimkien għal Ambjent Aħjar (FAA) said Friday ...,2024-12-13


In [6]:
sentiment_label = []

for index, row in df.iterrows():
    print(f'Headline: {row["headline"]}')
    print(f'Date: {row["date"]}')
    print(f'Description: {row["description"]}')
    print(f'URL: {row["url"]}')
    
    sentiment = pipe(row['description'])[0]
    print(f'Sentiment: {sentiment}')
    
    print('-' * 40)
    if sentiment['label'] == 'positive':
        total_score += sentiment['score']
        num_articles += 1
    elif sentiment['label'] == 'negative':
        total_score -= sentiment['score']
        num_articles += 1
        
    sentiment_label.append(sentiment['label'])
    df.at[index, 'sentiment'] = sentiment['label']
    df.at[index, 'sentiment_score'] = sentiment['score']
    
df.to_csv(fr'{file_path}', index=False)

final_score = total_score / num_articles if num_articles > 0 else 0


Headline: Inflation, environment main concerns for Maltese citizens – Eurobarometer
Date: 2024-12-19
Description: Inflation and the environment are the major concerns for Maltese citizens according to the latest Eurobarometer survey, published Thursday by the the European Commission Representation in Malta. The national report outlines the main findings of the fieldwork conducted between 10 and 30 October...
URL: https://maltabusinessweekly.com/inflation-environment-main-concerns-for-maltese-citizens-eurobarometer/27929/
Sentiment: {'label': 'neutral', 'score': 0.9277717471122742}
----------------------------------------
Headline: 16 local councils in financial trouble as they owe more money than they have
Date: 2024-12-19
Description: 16 local councils are in financial trouble as at the end of 2023 they owed more money than they had, a report by the National Audit Office has highlighted. The NAO said in a report that there were 16 local councils which ended the 2023 financial year wit